**Neighborhood Growth Model**

In [41]:
import numpy as np
import pandas as pd
import scipy as sp
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
import category_encoders as ce
import xgboost as xgb


In [2]:

# load all data
raw_data_t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1985.csv")
raw_data_t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")

# load the first quadrat that we will be working with
quad1_t1 = raw_data_t1.loc[raw_data_t1['quadrat']==1]
quad1_t2 = raw_data_t2.loc[raw_data_t2['quadrat']==1]




/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_43674/3378070634.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1985.csv")
/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_43674/3378070634.py:3: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")


In [3]:
# we only care about the tree IDs and DBHs of quad1_t2
expected_labels = quad1_t2[['treeID', 'dbh']]
expected_labels = expected_labels.rename(columns={"dbh": "dbh2", "treeID": "treeID2"})

#quad1_t1.head()
# simplify the data to have less features
quad1_t1 = quad1_t1[['treeID', 'sp', 'gx', 'gy', 'dbh']]
quad1_t1 = quad1_t1.rename(columns={"dbh": "dbh1", "treeID":"treeID1"})



In [4]:
encoder= ce.BinaryEncoder(cols=['sp'],return_df=True)
quad1_t1 = encoder.fit_transform(quad1_t1)
df_combined = pd.concat([quad1_t1, expected_labels], axis=1)

# Drop rows with any NaN values
df_combined_clean = df_combined.dropna()

# Now has more values because species is encoded to 7 different categories



In [5]:
# Separate the cleaned DataFrame and labels
df_clean = df_combined_clean[quad1_t1.columns]
labels_clean = df_combined_clean[expected_labels.columns]

quad1_t1 = df_clean.to_numpy()
expected_labels = labels_clean.to_numpy()

quad1_t1 = quad1_t1.astype(np.float32)
expected_labels = expected_labels.astype(np.float32)

In [6]:
ids = quad1_t1[:,0]
x_coordinates = quad1_t1[:, 8]  
y_coordinates = quad1_t1[:, 9]
coord_matrix = np.column_stack((x_coordinates, y_coordinates))
spatial_tree = sp.spatial.KDTree(coord_matrix)

In [35]:
nn_dist_matrix2 = np.zeros((len(coord_matrix),6))
nn_ind_matrix2 = np.zeros((len(coord_matrix),6))
nn_feats = np.column_stack((quad1_t1[:, 0:8], quad1_t1[:,10]))
feats_matrix = np.zeros((len(coord_matrix),54))

In [36]:
# NEW CODE AS OF JULY 6 - THIS WORKS!!!!!!!!

for i, tree in enumerate(coord_matrix):
    dist2, ind2 = spatial_tree.query(tree, k=6)
    nn_ind_matrix2[i] = ids[ind2]
    nn_dist_matrix2[i]= dist2

   
    nn_row = nn_feats[i].reshape(1,9)
    inc = 0
    for j in nn_ind_matrix2[i][1:]:
        row_ind = np.where(quad1_t1[:,0] == j)

        real_row = (quad1_t1[row_ind])
        distance = dist2[1:][inc].reshape(1,1)
        print (inc)
        print (distance)
        dbh = real_row[:,10].reshape(1,1)
        nn_row = np.hstack((nn_row, distance, real_row[:,1:8],dbh))
        inc += 1
    
    feats_matrix[i] = nn_row
    
nn_ind_matrix2 = nn_ind_matrix2[:,1:]
nn_dist_matrix2 = nn_dist_matrix2[:,1:]
# remove the columns with the focal tree's OWN distance and IDS



0
[[0.50000067]]
1
[[0.80000001]]
2
[[1.02956305]]
3
[[1.40356693]]
4
[[1.43178228]]
0
[[0.50990195]]
1
[[0.78102526]]
2
[[0.94868309]]
3
[[1.17047034]]
4
[[1.43178228]]
0
[[0.50990195]]
1
[[0.60827632]]
2
[[0.78102525]]
3
[[1.0198041]]
4
[[1.40356693]]
0
[[0.50000072]]
1
[[0.50990196]]
2
[[0.60827632]]
3
[[0.78102526]]
4
[[0.80000001]]
0
[[1.29999924]]
1
[[1.72626733]]
2
[[1.96468825]]
3
[[2.01246054]]
4
[[2.30867913]]
0
[[3.39558462]]
1
[[3.41760092]]
2
[[3.44383541]]
3
[[3.44818792]]
4
[[3.62215443]]
0
[[0.50000067]]
1
[[0.50000072]]
2
[[0.94339912]]
3
[[0.94868309]]
4
[[1.0198041]]
0
[[0.50990196]]
1
[[0.78102525]]
2
[[0.94339912]]
3
[[1.02956305]]
4
[[1.17047034]]
0
[[2.02484603]]
1
[[2.2825424]]
2
[[2.3769721]]
3
[[2.41867762]]
4
[[2.50599322]]
0
[[0.84852834]]
1
[[1.16618939]]
2
[[1.16619037]]
3
[[1.25299559]]
4
[[1.50000052]]
0
[[0.10000014]]
1
[[0.58309516]]
2
[[1.16618939]]
3
[[1.19999886]]
4
[[1.26490993]]
0
[[0.53851641]]
1
[[0.58309516]]
2
[[1.56524886]]
3
[[1.70293743]]
4

BELOW: trying to increase efficiency by using numpy broadcasting isntead of loops. will get back to this later on because it adds another dimension such that the data is nx1x50 instead of nx50.

In [9]:
#dist3, ind3 = tree2.query(coord_matrix, k=50)
#ind_array = np.arange(len(coord_matrix))

#nn_dist_matrix2 = dist3[ind_array[:,None],:]
#before = ids[ind3]
#nn_dist_matrix3 = before[ind_array[:,None],:]
#ADDS ANOTHER DIMENSION
#np.zeros((len(coord_matrix),50))
#nn_dist_matrix2[ind_array] = ids[ind3]

In [10]:
# change the labels so that they show GROWTH, not future DBH - better for readability of loss
expected_labels[:,1] = expected_labels[:,1] - quad1_t1[:,10]
expected_labels = np.where (expected_labels<0, 0, expected_labels)
X_train, X_test, y_train, y_test = train_test_split(feats_matrix, expected_labels, test_size=0.3)

feats = X_train[:,1:]
labels = y_train[:,1]

test_ids = X_test[:,0]
test_feats = X_test[:,1:]

Using SKLearn's Random Forest Regressor

In [11]:
random_forest = RandomForestRegressor()
random_forest.fit(feats,labels)

preds = random_forest.predict(test_feats)
preds_matrix = np.column_stack((test_ids, preds))

error1 = sklearn.metrics.mean_squared_error(y_test[:,1], preds)
print (np.sqrt(error1))
error1a = r2_score(y_test[:,1],preds)
print (error1a)

13.073445459804244
0.4258394509309078


Using XGBoost Regressor

In [13]:
xgb_tree = xgb.XGBRegressor()
xgb_tree.fit(feats,labels)
preds2 = xgb_tree.predict(test_feats)
predictions_matrix = np.column_stack((test_ids, preds2))
error2 = sklearn.metrics.mean_squared_error(y_test[:,1], preds2)
print (np.sqrt(error2))
error2a = r2_score (y_test[:,1], preds2)
print (error2a)

11.779322
0.5338842293710324


Using K Folds Cross Validation to Introduce Randomness!

In [45]:
cv = KFold(n_splits=5)
cv_rf = RandomForestRegressor()

scores = cross_validate(random_forest, feats_matrix, expected_labels, scoring=('neg_root_mean_squared_error','r2'), cv=cv)

print (scores['test_neg_root_mean_squared_error'])
print (scores['test_r2'])

[-2.10575969e+04 -8.48510113e+02 -1.36802842e+01 -2.27794534e+02
 -1.82693303e+03]
[-6.14023072e+00 -1.13172544e+04 -1.28775923e+00 -6.01565721e+02
 -1.75445937e+00]
